**SQL Alchemy Quickstart**

In [1]:
from sqlalchemy import create_engine

# Example for SQLite (creates a local file)
DATABASE_URL = "sqlite:///./test.db"
engine = create_engine(DATABASE_URL, echo=True) # `echo=True` logs generated SQL to the console
engine

Engine(sqlite:///./test.db)

**Simple tool call to retrive data from SQL database**

This is done using langchain tool, the db was downloaded from course repo. This is used for learning purposes. 

In [2]:
from langchain_community.utilities import SQLDatabase
db = SQLDatabase.from_uri("sqlite:///resources/Chinook.db")

In [ ]:
from langchain.tools import tool    

@tool
def sql_query(query: str) -> str:
    """Obtain information from the database using SQL queries, send the arguments as a string"""

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"
    
sql_query.invoke("SELECT * FROM Artist LIMIT 3")

"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith')]"

In [7]:
# Load the model

from langchain.agents import create_agent
from langchain_ollama import ChatOllama

# Load the model
model = ChatOllama(model="llama3.2:1b", 
                   temperature=0,
                   validate_model_on_init=True,
                   seed=42)

agent = create_agent(
    model,
    tools=[sql_query])

In [8]:
from langchain.messages import HumanMessage

question = HumanMessage(content="Who is the most popular artist beginning with 'S' in this database?")

response = agent.invoke(
    {"messages": [question]}
)

In [9]:
response

{'messages': [HumanMessage(content="Who is the most popular artist beginning with 'S' in this database?", additional_kwargs={}, response_metadata={}, id='f101dcf7-6bb7-48f6-907e-055ce3cdf68b'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-12-27T10:19:16.7909624Z', 'done': True, 'done_reason': 'stop', 'total_duration': 72110767500, 'load_duration': 71017381000, 'prompt_eval_count': 167, 'prompt_eval_duration': 604803400, 'eval_count': 30, 'eval_duration': 405161300, 'logprobs': None, 'model_name': 'llama3.2:1b', 'model_provider': 'ollama'}, id='lc_run--019b5f50-b9fe-7872-8d4e-6e3b50be2395-0', tool_calls=[{'name': 'sql_query', 'args': {'query': 'SELECT * FROM artists WHERE first_name LIKE %s'}, 'id': 'f7374d6a-8d1c-46cf-af04-d305980c99ab', 'type': 'tool_call'}], usage_metadata={'input_tokens': 167, 'output_tokens': 30, 'total_tokens': 197}),
  ToolMessage(content='Error: (sqlite3.OperationalError) near "%": syntax error\n[SQ

In [15]:
print(response["messages"][-3].tool_calls[0]['args']['query'])

SELECT * FROM artists WHERE first_name LIKE %s


### Semantic Search

Perform semantic serach in a PDF docuemnt using langchain document loader class. 

In [1]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("resources/acmecorp-employee-handbook.pdf")

data = loader.load()

print(data)

[Document(metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2025-11-20T23:23:16+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2025-11-20T23:23:16+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': 'resources/acmecorp-employee-handbook.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='Employee Handbook\nNon-Disclosure Agreement (NDA) Policy\nEmployees must protect confidential information belonging to the company, its clients, and partners.\nThis includes, but is not limited to, product roadmaps, customer data, internal communications,\nproprietary algorithms, financial information, and unreleased features. Confidential information may not\nbe shared with unauthorized individuals inside or outside the organization. These obligations continue\nafter employment ends.\nWorkplace Conduct Policy\nEmployees must maintain a respectful, professional environment

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(data)

print(len(all_splits))

3


In [3]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="snowflake-arctic-embed:22m",
)

In [5]:
from langchain_core.vectorstores import InMemoryVectorStore
vector_store = InMemoryVectorStore(embeddings)

# Adding the document splits into the vector store 
ids = vector_store.add_documents(documents=all_splits)

In [6]:
results = vector_store.similarity_search(
    "How many days of vacation does an employee get in their first year?"
)

print(results[0])

page_content='business travel. This includes transportation, lodging, meals, and incidental expenses within
established limits. Receipts must be submitted within 14 days of travel. First-class travel, personal
expenses, and non-business activities are not reimbursable. Employees should exercise good
judgment and cost-effective decision-making when traveling on behalf of the company.' metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2025-11-20T23:23:16+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2025-11-20T23:23:16+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': 'resources/acmecorp-employee-handbook.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1', 'start_index': 1571}


That's neat

### RAG Agent

In [9]:
from langchain.tools import tool
@tool
def search_handbook(query: str) -> str:
    """Search the employee handbook for information"""
    results = vector_store.similarity_search(query)
    return results[0].page_content

In [10]:
from langchain.agents import create_agent
from langchain_ollama import ChatOllama
# Load the model
model = ChatOllama(model="llama3.2:1b", 
                   temperature=0,
                   validate_model_on_init=True,
                   seed=42)

agent = create_agent(
    model,
    tools=[search_handbook],
    system_prompt="You are a helpful agent that can search the employee handbook for information."
    )

In [ ]:
from langchain.messages import HumanMessage
from pprint import pprint
response = agent.invoke(
    {"messages": [HumanMessage(content="How many days of vacation does an employee get in their first year?")]}
)

pprint(response)

{'messages': [HumanMessage(content='How many days of vacation does an employee get in their first year?', additional_kwargs={}, response_metadata={}, id='fa1776fc-3a99-4192-b183-2528c8987f95'),
              AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-12-27T11:48:48.2891198Z', 'done': True, 'done_reason': 'stop', 'total_duration': 846361300, 'load_duration': 197469700, 'prompt_eval_count': 177, 'prompt_eval_duration': 367165100, 'eval_count': 27, 'eval_duration': 235186200, 'logprobs': None, 'model_name': 'llama3.2:1b', 'model_provider': 'ollama'}, id='lc_run--019b5fa3-c6d0-77b1-b619-ae50fa4f594d-0', tool_calls=[{'name': 'search_handbook', 'args': {'query': 'vacation days in first year'}, 'id': 'f746486a-4400-482c-ae2d-94ae07dac6af', 'type': 'tool_call'}], usage_metadata={'input_tokens': 177, 'output_tokens': 27, 'total_tokens': 204}),
              ToolMessage(content='prohibited. Violations may result in disciplinary acti

In [14]:
pprint(response['messages'][-1].content)

('Based on the search results, it appears that full-time employees accrue 10 '
 'days of paid vacation per year in their first year of service.\n'
 '\n'
 "Here's a formatted answer to your original question:\n"
 '\n'
 '"According to our company\'s PTO policy, new hires receive 10 days of paid '
 'vacation in their first year of employment."')
